In [1]:
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k):
        print("Getattr of parent callback called")
        return getattr(self.run, k)
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')

In [98]:
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        #print(dir(self))
        self.run.n_epochs=0
        print(self.run,"Hello")
        print(self.a)
        print(self.run.model)
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

In [99]:
from typing import *

def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, Iterable): return list(o)
    return [o]

In [114]:

class Runner:
    """
    Contains the one_batch,all_batches and fit function
    """
    
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            print(cbf.__class__.__name__)
            print(cbf)
            cb = cbf()
            print(cb.name)
            setattr(self, cb.name, cb)
            cbs.append(cb)
        
        self.stop,self.cbs = False,[TrainEvalCallback()]+cbs
    
    @property
    def model(self):     return "This is a model"
        
    def __call__(self, cb_name):
        for cb in sorted(self.cbs, key=lambda x: x._order):
            f = getattr(cb, cb_name, None)
            print("callable")
            print(f)
            if f and f(): return True
        return False
    
    def fit(self):
        #print(self.cbs)
        for cb in self.cbs:
            cb.set_runner(self)
            self.a = "Abhinav"
            if not self("begin_epoch"):
                print(cb.n_epochs,"n_epochs")
            print(self.n_epochs)
                
            
        

In [100]:
run = Runner()

In [101]:

run.fit()

callable
<bound method TrainEvalCallback.begin_epoch of <__main__.TrainEvalCallback object at 0x10f2d12e8>>
<__main__.Runner object at 0x10f2d1b38> Hello
Getattr of parent callback called
Abhinav
This is a model
Getattr of parent callback called
0 n_epochs
0


### Doubt on partials applied to classes clarified

In [102]:
class AvgStatsCallback(Callback):
    def __init__(self, metrics):
        self.train_stats,self.valid_stats = metrics,metrics
    

In [104]:
from functools import partial
acc = partial(AvgStatsCallback,"metrics")

In [106]:
acc().train_stats

'metrics'

In [115]:
run1 = Runner(cb_funcs=acc)

partial
functools.partial(<class '__main__.AvgStatsCallback'>, 'metrics')
avg_stats


In [119]:
run1.avg_stats.train_stats

'metrics'

In [121]:
type(acc())

__main__.AvgStatsCallback

In [122]:
type(run1)

__main__.Runner